In [1]:
###########################################
# 场景复现，AHP建模
# 可以从以下五个方面来对网络攻击进行评估，分别是
# 1、探测信息
# 2、拒绝服务
# 3、破坏数据
# 4、对抗防御
# 5、入侵控制
###########################################

import numpy as np
from copy import deepcopy
import operator

n = 5

###############################
# 以下是六位专家给出的对比矩阵
###############################

A = np.round([[1,1,4,5,2],[1,1,6,7,2],[1/4,1/6,1,1/2,1/5],[1/5,1/7,2,1,1/6],[1/2,1/2,5,6,1]],decimals=3)
B = np.round([[1,1/7,1/3,1/5,1/2],[7,1,6,3,4],[3,1/6,1,1/3,1/3],[5,1/3,3,1,3],[2,1/4,3,1/3,1]],decimals=3)
C = np.round([[1,1/9,1/3,1/7,1/3],[9,1,7,3,4],[3,1/7,1,1/5,1/2],[7,1/3,5,1,2],[1/3,1/4,2,1/2,1]],decimals=3)
D = np.round([[1,1/9,1/4,1/7,1/2],[9,1,8,3,7],[4,1/8,1,1/5,1],[7,1/3,5,1,3],[2,1/7,1,1/3,1]],decimals=3)
E = np.round([[1,1/9,1/3,1/7,1/3],[9,1,7,3,4],[3,1/7,1,1/5,1/2],[6,1/3,5,1,2],[3,1/4,2,1/2,1]],decimals=3)
F = np.round([[1,1/9,1/3,1/6,1/3],[9,1,7,3,4],[3,1/7,1,1/5,1/2],[6,1/3,5,1,2],[1/3,1/4,2,1/2,1]],decimals=3)

expert_opinion = [A,B,C,D,E,F]
expert_opinion_origin = deepcopy(expert_opinion)
expert_name = ['A','B','C','D','E','F']


In [2]:
#############################
# 将特征矩阵的每一列进行归一化操作
#############################
for i in range(len(expert_opinion)):
    columns_sum = np.cumsum(expert_opinion[i],axis=0)[n-1]
    for row in range(len(expert_opinion[i])):
        expert_opinion[i][row] = expert_opinion[i][row] / columns_sum

In [3]:
####################################
# 分别对每一个专家给出的对比矩阵计算权向量
####################################
expert_vector = []
for e in expert_opinion:
    row_sum = np.cumsum(e,axis=1)[:,n-1]
    expert_vector.append(row_sum / n)
    
expert_vector

[array([0.30922702, 0.35196206, 0.05252754, 0.06243911, 0.22384428]),
 array([0.05075602, 0.48730645, 0.08720437, 0.24476754, 0.12996562]),
 array([0.04067264, 0.51472769, 0.07914715, 0.26268988, 0.10276264]),
 array([0.03905797, 0.54031613, 0.08705449, 0.2561168 , 0.07745461]),
 array([0.03992732, 0.50801983, 0.0769112 , 0.24838175, 0.1267599 ]),
 array([0.04213853, 0.51869578, 0.08063276, 0.25570269, 0.10283024])]

In [4]:
expert_opinion_origin

[array([[1.   , 1.   , 4.   , 5.   , 2.   ],
        [1.   , 1.   , 6.   , 7.   , 2.   ],
        [0.25 , 0.167, 1.   , 0.5  , 0.2  ],
        [0.2  , 0.143, 2.   , 1.   , 0.167],
        [0.5  , 0.5  , 5.   , 6.   , 1.   ]]),
 array([[1.   , 0.143, 0.333, 0.2  , 0.5  ],
        [7.   , 1.   , 6.   , 3.   , 4.   ],
        [3.   , 0.167, 1.   , 0.333, 0.333],
        [5.   , 0.333, 3.   , 1.   , 3.   ],
        [2.   , 0.25 , 3.   , 0.333, 1.   ]]),
 array([[1.   , 0.111, 0.333, 0.143, 0.333],
        [9.   , 1.   , 7.   , 3.   , 4.   ],
        [3.   , 0.143, 1.   , 0.2  , 0.5  ],
        [7.   , 0.333, 5.   , 1.   , 2.   ],
        [0.333, 0.25 , 2.   , 0.5  , 1.   ]]),
 array([[1.   , 0.111, 0.25 , 0.143, 0.5  ],
        [9.   , 1.   , 8.   , 3.   , 7.   ],
        [4.   , 0.125, 1.   , 0.2  , 1.   ],
        [7.   , 0.333, 5.   , 1.   , 3.   ],
        [2.   , 0.143, 1.   , 0.333, 1.   ]]),
 array([[1.   , 0.111, 0.333, 0.143, 0.333],
        [9.   , 1.   , 7.   , 3.   , 4.   ],
  

In [5]:
###########
# 一致性检验
###########
RI = {2:0,3:0.58,4:0.90,5:1.12,6:1.24,7:1.32,8:1.41,9:1.45}
Aw = []
for index in range(len(expert_opinion_origin)):
    Aw.append(expert_opinion_origin[index] @ expert_vector[index])
print('Aw: ',Aw)
answer_max = []
for index in range(len(Aw)):
    answer_max.append(np.sum(Aw[index] / expert_vector[index]) / n)
answer_max = np.array(answer_max)
print('answer_max: ',answer_max)
CI = (answer_max - n) / (n - 1)
CIRI = CI/RI[n]
#########################################################
# 可以看出，每个专家的一致性检验的结果都低于0.1，在可以接受的范围内
#########################################################
print("CIRI: ",CIRI)

Aw:  [array([1.63118333, 1.86111663, 0.26460037, 0.31705216, 1.19171117]), array([0.26341621, 2.6199899 , 0.44563874, 1.31233065, 0.69642497]), array([0.19594803, 2.63393169, 0.37869043, 1.3200637 , 0.53462779]), array([0.19614869, 2.89880642, 0.43950385, 1.37708412, 0.40517714]), array([0.19965859, 2.65792895, 0.3823963 , 1.29519207, 0.65156009]), array([0.20350929, 2.6408009 , 0.38377751, 1.29008385, 0.53565318])]
answer_max:  [5.20037091 5.27933513 4.98943791 5.20871044 5.11180871 4.98693732]
CIRI:  [ 0.04472565  0.06235159 -0.00235761  0.04658715  0.0249573  -0.00291578]


In [6]:
############
# 结果展示
############

import pandas as pd

df = pd.DataFrame(expert_vector,columns=['w1','w2','w3','w4','w5'],index=['A','B','C','D','E','F'])
df['CI'] = CI
df['CI/RI'] = CIRI
df['一致性'] = df['CI/RI'].map(lambda x: x < 0.1)

In [7]:
df
##############################################
# 如果有专家意见的一致性不符合要求，需要重新写对比矩阵
##############################################

,w1,w2,w3,w4,w5,CI,CI/RI,一致性
A,0.309227,0.351962,0.052528,0.062439,0.223844,0.050093,0.044726,True
B,0.050756,0.487306,0.087204,0.244768,0.129966,0.069834,0.062352,True
C,0.040673,0.514728,0.079147,0.262690,0.102763,-0.002641,-0.002358,True
D,0.039058,0.540316,0.087054,0.256117,0.077455,0.052178,0.046587,True
E,0.039927,0.508020,0.076911,0.248382,0.126760,0.027952,0.024957,True
F,0.042139,0.518696,0.080633,0.255703,0.102830,-0.003266,-0.002916,True


In [8]:
#######################
# 获得每个专家的权重排序
#######################
columns_name = []
for i in range(1,n + 1):
    columns_name.append('w' + str(i))
print(columns_name)

expert_res = {}
for expert in expert_name:
    tmp = []
    for i in sorted(df.loc[expert,columns_name].to_dict().items(),key=operator.itemgetter(1),reverse=True):
        tmp.append(i[0])
    expert_res[expert] = '>'.join(tmp)
print(expert_res)

['w1', 'w2', 'w3', 'w4', 'w5']
{'A': 'w2>w1>w5>w4>w3', 'B': 'w2>w4>w5>w3>w1', 'C': 'w2>w4>w5>w3>w1', 'D': 'w2>w4>w3>w5>w1', 'E': 'w2>w4>w5>w3>w1', 'F': 'w2>w4>w5>w3>w1'}


In [9]:
##################################################
# 在专家意见中取出最一致的答案，并取出给出相对应意见的专家
##################################################
from collections import Counter
res = sorted(Counter(expert_res.values()).items(),key=operator.itemgetter(1),reverse=True)[0][0]
print(res)
choosed_expert = []
for key,value in expert_res.items():
    if value == res:
        choosed_expert.append(key)
choosed_expert

w2>w4>w5>w3>w1


['B', 'C', 'E', 'F']

In [10]:
#############
# 最终结果如下
#############
final_res = []
for cname in columns_name:
    final_res.append(np.power(np.prod(df.loc[choosed_expert,cname]),1/len(choosed_expert)))
final_res

[0.04317028080130074,
 0.5070412231731407,
 0.08088512363642572,
 0.2527918304529095,
 0.11486603138637902]

In [11]:
########################################################
# 最后一步的最后一步（可选），设定一个阈值a，对得到的权重进行筛选。
# 如果w > a,则保留该属性，否则删除.
# 将忽略的属性权重求和，平均分配到各个保留属性系数上
########################################################
threshold = 0.05
bt_threshold = []
lt_threshold = []
for r in final_res:
    if r >= threshold:
        bt_threshold.append(r)
    else:
        lt_threshold.append(r)
print('bt_threshold: ',bt_threshold)
print('lt_threshold: ',lt_threshold)

bt_threshold:  [0.5070412231731407, 0.08088512363642572, 0.2527918304529095, 0.11486603138637902]
lt_threshold:  [0.04317028080130074]


In [12]:
for index in range(len(bt_threshold)):
    bt_threshold[index] += sum(lt_threshold) / len(bt_threshold)
print('bt_threshold: ',bt_threshold)

bt_threshold:  [0.5178337933734659, 0.09167769383675091, 0.2635844006532347, 0.1256586015867042]
